# Análisis inicial y trata de datos

In [ ]:
# set PYTHONPATH="${PYTHONPATH}:/path/to/your/project/"

In [2]:
# Importamos las librerías necesarias
import pandas as pd # type: ignore
import numpy as np # type: ignore
# from src import funciones_soporte as fs
import psycopg2 as ps # type: ignore

In [3]:
# Importamos el archivo con el que vamos a trabajar, que en este caso es un parquet. Para que funcione el read_parquet es necesario pasarle
# el parámetro ' engine = auto/pyarrow', ambos funcionan. En caso de que se use pyarrow será necesario realizar su instalaciñon previamnente en el entorno.
df_raw = pd.read_parquet("../data/reservas_hoteles.parquet", engine='auto')
df_raw.head()

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,,2025-03-01,2025-03-02,113,NaN,,NaN,
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,,2025-03-01,2025-03-02,194,NaN,,NaN,
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,6ec4d7d2-e382-47cc-94f6-c647577d4035,Maite,Calatayud,maite.calatayud@example.com,False,2025-02-09,2025-03-01,2025-03-02,14,119.96,Hotel Monte Verde,1.0,Madrid
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,aed17fe8-eb18-4f69-bf08-df1afdc339c3,Tecla,Bonet,tecla.bonet@example.com,False,2025-02-08,2025-03-01,2025-03-02,38,211.53,Hotel Brisas del Mar,5.0,Madrid
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,710a2fbb-264f-426a-aa66-f8fffd146a2d,Amílcar,Andrés,amílcar.andrés@example.com,False,2025-02-02,2025-03-01,2025-03-02,35,378.37,Hotel Camino del Sol,1.0,Madrid


In [4]:
# Creamos una copia de los datos originales
data = df_raw.copy()
data.head()

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,,2025-03-01,2025-03-02,113,NaN,,NaN,
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,,2025-03-01,2025-03-02,194,NaN,,NaN,
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,6ec4d7d2-e382-47cc-94f6-c647577d4035,Maite,Calatayud,maite.calatayud@example.com,False,2025-02-09,2025-03-01,2025-03-02,14,119.96,Hotel Monte Verde,1.0,Madrid
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,aed17fe8-eb18-4f69-bf08-df1afdc339c3,Tecla,Bonet,tecla.bonet@example.com,False,2025-02-08,2025-03-01,2025-03-02,38,211.53,Hotel Brisas del Mar,5.0,Madrid
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,710a2fbb-264f-426a-aa66-f8fffd146a2d,Amílcar,Andrés,amílcar.andrés@example.com,False,2025-02-02,2025-03-01,2025-03-02,35,378.37,Hotel Camino del Sol,1.0,Madrid


In [5]:
# función info
def info_df(dataframe):
    """
    Función que devuelve información general sobre el DatFrame que le pasemos.


    Args:
        df (DataFrame): DataFrame con información que queramos revisar

    Returns:
        DataFrame: DataFrame con información general sobre las columnas del DataFrame que se le ha pasado a la función: tipo de datos, número de
        registros, número de valores nulos, porcentaje de los valores nulos sobre el total
    """
    info_df = pd.DataFrame()
    info_df["Tipo_dato"] = dataframe.dtypes
    info_df["numero_registros"] = [dataframe[elemento].value_counts().sum() for elemento in dataframe]
    info_df["Numero_nulos"] = round(dataframe.isnull().sum())
    info_df["%_nulos"] = round((dataframe.isnull().sum()/dataframe.shape[0])*100, 2)

    return info_df

In [6]:
# Información general del dataframe
info_df(data)

,Tipo_dato,numero_registros,Numero_nulos,%_nulos
id_reserva,object,15098,0,0.00
id_cliente,object,15098,0,0.00
nombre,object,15098,0,0.00
apellido,object,15098,0,0.00
mail,object,15098,0,0.00
competencia,bool,15098,0,0.00
fecha_reserva,object,15098,0,0.00
inicio_estancia,object,15023,75,0.50
final_estancia,object,15023,75,0.50
id_hotel,int64,15098,0,0.00


In [7]:
# Vamos a ver las columnas que tenemos en el dataframe
data.columns

Index(['id_reserva', 'id_cliente', 'nombre', 'apellido', 'mail', 'competencia',
       'fecha_reserva', 'inicio_estancia', 'final_estancia', 'id_hotel',
       'precio_noche', 'nombre_hotel', 'estrellas', 'ciudad'],
      dtype='object')

Tras realizar una overview de los datos recibidos observamos las siguientes casuísticas:
- Tenemos 14 columnas.
- Hay 3 columnas con información sobre fechas, las cuales son de tipo object. Será necesario modificar el tipo de dato de las mismas para poder trabajar más adelante.
- La columna competencia es un booleano, el cual habrá que ver con que se corresponde true y false.
- Tenemos tres columnas de id: de reserva, de cliente y de hotel. Las dos primeras con tipo object, y la última con tipo int. Cambiaremos la de id_hotel a tipo object para tenerlas las tres iguales. 
- Hay existencia de nulos en cuatro columnas:
    
    **a)** inicio_estancia: tenemos un 0,5% de nulos.
    
    **b)** fin_estancia: tenemos un 0,5% de nulos. Comprobaremos si son los mismos que la columna de inicio estancia, revisar casuística y tratar.
    
    **c)** precio_noche: hay un 34,6% de nulos, un valor significativo. Será necesario tratarlos.
    
    **d)** estrellas: hay un 34,26% de nulos, un valor significativo. Será necesario tratarlos. Asimismo el tipo de dato de esta columna es float. Podemos plantear modificarlo a tipo object, a un str. 

A continuación realizamos las modificaciones necesarias de las columnas para poder trabajar con ellas. 

## Modificación columnas de fecha

In [8]:
# Creamos una función que nos modifica ek tipo de dato de las columnas que le pasemos a tipo fecha
def data_fechas(dataframe, columnas):
    for col in columnas:
        dataframe[col] = pd.to_datetime(dataframe[col])
    return dataframe

In [9]:
# modificamos las columnas que queremos a tipo fecha
data = data_fechas(data, ["fecha_reserva", "inicio_estancia", "final_estancia"])

In [10]:
# comprobamos que la modificación se ha realizado de forma correcta
info_df(data)

,Tipo_dato,numero_registros,Numero_nulos,%_nulos
id_reserva,object,15098,0,0.00
id_cliente,object,15098,0,0.00
nombre,object,15098,0,0.00
apellido,object,15098,0,0.00
mail,object,15098,0,0.00
competencia,bool,15098,0,0.00
fecha_reserva,datetime64[ns],9926,5172,34.26
inicio_estancia,datetime64[ns],15023,75,0.50
final_estancia,datetime64[ns],15023,75,0.50
id_hotel,int64,15098,0,0.00


Modificación realizada con éxito.

## Columna valores booleanos

In [11]:
data.head()

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,NaT,2025-03-01,2025-03-02,113,NaN,,NaN,
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,NaT,2025-03-01,2025-03-02,194,NaN,,NaN,
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,6ec4d7d2-e382-47cc-94f6-c647577d4035,Maite,Calatayud,maite.calatayud@example.com,False,2025-02-09,2025-03-01,2025-03-02,14,119.96,Hotel Monte Verde,1.0,Madrid
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,aed17fe8-eb18-4f69-bf08-df1afdc339c3,Tecla,Bonet,tecla.bonet@example.com,False,2025-02-08,2025-03-01,2025-03-02,38,211.53,Hotel Brisas del Mar,5.0,Madrid
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,710a2fbb-264f-426a-aa66-f8fffd146a2d,Amílcar,Andrés,amílcar.andrés@example.com,False,2025-02-02,2025-03-01,2025-03-02,35,378.37,Hotel Camino del Sol,1.0,Madrid


In [16]:
# Al intentar filtrar la columna en sí, como es un booleano no nos devuelve nada, por lo que creamos una copia de la columna de tipo str
# a través de la cual podremos realizar el filtrado entre true y false.
data["col_prueba"] = data["competencia"].astype(str)
data.head()

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad,col_prueba
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,NaT,2025-03-01,2025-03-02,113,NaN,,NaN,,True
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,NaT,2025-03-01,2025-03-02,194,NaN,,NaN,,True
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,6ec4d7d2-e382-47cc-94f6-c647577d4035,Maite,Calatayud,maite.calatayud@example.com,False,2025-02-09,2025-03-01,2025-03-02,14,119.96,Hotel Monte Verde,1.0,Madrid,False
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,aed17fe8-eb18-4f69-bf08-df1afdc339c3,Tecla,Bonet,tecla.bonet@example.com,False,2025-02-08,2025-03-01,2025-03-02,38,211.53,Hotel Brisas del Mar,5.0,Madrid,False
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,710a2fbb-264f-426a-aa66-f8fffd146a2d,Amílcar,Andrés,amílcar.andrés@example.com,False,2025-02-02,2025-03-01,2025-03-02,35,378.37,Hotel Camino del Sol,1.0,Madrid,False


Hoteles de la competencia (col True)

Estas columnas entendemos que se consideran como competencia del grupo, por lo que nos falta información de las mismas.

Tenemos valores nulos en este caso en las columnas de inicio y fin de estancia (23 en cada una), los cuales se corresponden en ambas columnas con los mismos clientes. Asimismo hay valores nulos en la columna de precio de noche (5172), que entendemos que se corresponde con los valores nulos de los hoteles de los cuales no tenemos estrellas (nulos en columna estrellas.)

Para tratar estos datos realizaremos un scapeo de los nombres de los hoteles correspondientes, sus estrellas y el precio por noche de cada uno. Buscaremos realizar este scrapeo en función del id de hotel, ya que los id los tenemos.

Tenemos valores duplicados en la columna de id de cliente, será necesario eliminar estos valores. 

In [41]:
# Filtramos por los valores que son True
df_true = data[data["col_prueba"] == "True"]
df_true.head()

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad,col_prueba
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,NaT,2025-03-01,2025-03-02,113,NaN,,NaN,,True
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,NaT,2025-03-01,2025-03-02,194,NaN,,NaN,,True
7,4d65096e-5f12-4ce7-9fd6-dd20e3cf5194,443a7307-2183-4051-b0bb-26f12fc1f478,Judith,Rivas,judith.rivas@example.com,True,NaT,2025-03-01,2025-03-02,131,NaN,,NaN,,True
9,458029ad-cc9b-4d85-8290-fec170bcc02d,e1a34338-a0aa-4b0e-96de-55f884eeb074,Adolfo,Cuervo,adolfo.cuervo@example.com,True,NaT,2025-03-01,2025-03-02,114,NaN,,NaN,,True
11,ac0fe86e-90a0-4fe6-a02e-6ea00e9a8621,2949fb90-39ab-4613-b4c1-e238bb56faf3,María Carmen,Navarro,maría carmen.navarro@example.com,True,NaT,2025-03-01,2025-03-02,103,NaN,,NaN,,True


In [51]:
# Vamos a revisar lso valores nulos en la columna de inicio de estancia y final de estancia
# print(df_true[df_true["inicio_estancia"].isnull().sum()])
# print(df_true[df_true["final_estancia"].isnull().sum()])

In [ ]:
# Comprobamos si los valores nulos de las columnas de inicio y final de estancia se corresponden con los mismos clientes en ambas columnas.
# df_true[df_true["inicio_estancia"].isnull()] == df_true[df_true["final_estancia"].isnull()]

In [81]:
# df_true[df_true["precio_noche"].isnull().sum()]

In [82]:
# df_true[df_true["estrellas"].isnull().sum()]

In [ ]:
# Comprobamos los nombres de los hoteles de la competencia
df_true["nombre_hotel"].unique()

array([''], dtype=object)

In [ ]:
# Comprobamos los id de hotel de la competencia
df_true["id_hotel"].unique()

array([113, 194, 131, 114, 103, 181, 128, 186, 135, 117])

In [73]:
df_true.duplicated().sum()

np.int64(0)

In [ ]:
# Revisamos duplicados
df_true["id_cliente"].duplicated().sum()

np.int64(11)

Hoteles del grupo (col False)

Estas columnas entendemos que no se consideran como competencia del grupo, por lo que tenemos casi toda la información de las mismas. Tenemos los nombres de los hoteles del grupo, junto a sus id correspondientes.

Tenemos valores nulos en este caso en las columnas de inicio, fin de estancia y precio de noche (52 en cada una), los cuales se corresponden en todas las columnas con los mismos clientes.

Tenemos duplicados en las columnas de id de cliente (178) e id de reserva (98)

In [ ]:
# Filtramos por las columnas que contienen False.
df_false = data[data["col_prueba"] == "False"]
df_false.head()

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad,col_prueba
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,6ec4d7d2-e382-47cc-94f6-c647577d4035,Maite,Calatayud,maite.calatayud@example.com,False,2025-02-09,2025-03-01,2025-03-02,14,119.96,Hotel Monte Verde,1.0,Madrid,False
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,aed17fe8-eb18-4f69-bf08-df1afdc339c3,Tecla,Bonet,tecla.bonet@example.com,False,2025-02-08,2025-03-01,2025-03-02,38,211.53,Hotel Brisas del Mar,5.0,Madrid,False
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,710a2fbb-264f-426a-aa66-f8fffd146a2d,Amílcar,Andrés,amílcar.andrés@example.com,False,2025-02-02,2025-03-01,2025-03-02,35,378.37,Hotel Camino del Sol,1.0,Madrid,False
5,ccbd9fe9-5a60-4f6f-bed3-73a7158ecba7,df42ae3d-0b9b-4c06-b22e-939323833ef8,Joan,Vazquez,joan.vazquez@example.com,False,2025-02-08,2025-03-01,2025-03-02,36,85.45,Hotel Puerta del Cielo,5.0,Madrid,False
6,99c3dc4f-663c-45f7-849a-ac9313f3746a,a16a9d83-6888-4947-a7ab-ed41203e347c,Chelo,Flor,chelo.flor@example.com,False,2025-02-09,2025-03-01,2025-03-02,43,309.80,Hotel Encanto Real,1.0,Madrid,False


In [63]:
# Vamos a revisar lso valores nulos en la columna de inicio de estancia y final de estancia
# df_false[df_false["inicio_estancia"].isnull()] 
# df_false[df_false["final_estancia"].isnull()]
# df_false[df_false["precio_noche"].isnull()]

In [38]:
# Comprobamos si los valores nulos de las columnas de inicio y final de estancia se corresponden con los mismos clientes en ambas columnas.
# df_false[df_false["inicio_estancia"].isnull()] == df_false[df_false["final_estancia"].isnull()]

In [40]:
# Vamos a revisar los nombres de los hoteles y su id_correspondiente
df_false[["id_hotel","nombre_hotel"]]

,id_hotel,nombre_hotel
2,14,Hotel Monte Verde
3,38,Hotel Brisas del Mar
4,35,Hotel Camino del Sol
5,36,Hotel Puerta del Cielo
6,43,Hotel Encanto Real
...,...,...
15093,36,Hotel Las Estrellas
15094,38,Gran Hotel Madrid
15095,29,Hotel Jardines del Rey
15096,15,Hotel Palacio Imperial


In [72]:
df_false.duplicated().sum()

np.int64(98)

In [ ]:
# Revisamos duplicados de los valores 
df_false["id_reserva"].duplicated().sum()

np.int64(98)

In [71]:
df_false["id_cliente"].duplicated().sum()

np.int64(178)

## Columna id_hotel

In [ ]:
# Tenemos el id del hotel como tipo int, pero nos interesa modificarlo a tipo str/object
data["id_hotel"].dtypes

dtype('int64')

In [ ]:
# Realizamos la modificación del tipo de dato
data["id_hotel"] = data["id_hotel"].astype(str)

In [ ]:
# Comprobamos que se ha realizado la modificación correctamente
info_df(data)

,Tipo_dato,numero_registros,Numero_nulos,%_nulos
id_reserva,object,15098,0,0.00
id_cliente,object,15098,0,0.00
nombre,object,15098,0,0.00
apellido,object,15098,0,0.00
mail,object,15098,0,0.00
competencia,bool,15098,0,0.00
fecha_reserva,datetime64[ns],9926,5172,34.26
inicio_estancia,datetime64[ns],15023,75,0.50
final_estancia,datetime64[ns],15023,75,0.50
id_hotel,object,15098,0,0.00
